# Hartmann Hahn polarising sequence

In [2]:
from collections import OrderedDict
import datetime
import numpy as np
import time
import os

try: 
    pulsedmasterlogic
except NameError:
    manager.startModule('logic', 'pulsedmasterlogic')
try: 
    scannerlogic
except NameError:
    manager.startModule('logic', 'scannerlogic')
try: 
    optimizer
except NameError:
    manager.startModule('logic', 'optimizer')
try:
    sequencegenerator
except NameError:
    manager.startModule('logic', 'sequencegenerator')
try:
    pulsedmeasurementlogic
except NameError:
    manager.startModule('gui', 'pulsedmeasurementlogic')   
try:
    poimanager
except NameError:
    manager.startModule('logic', 'poimaganer')  
    
def write_to_logfile(nametag, timestamp, name, **kwargs):
    """ Write parameters to custom logfile with name nametag """
    if type(timestamp) is not str:
        timestamp = str(timestamp)
    parameters = list(kwargs)
    if len(parameters) == 1 and type(kwargs[parameters[0]]) is OrderedDict:
        param_dict = kwargs[parameters[0]]
        parameters = list(param_dict)
        kwargs = param_dict
    log_dir = pulsedmeasurementlogic.savelogic().get_path_for_module('CustomLogfiles')
    log_path = os.path.join(log_dir, nametag + '.txt')
    if not os.path.isfile(log_path):
        with open(log_path, 'w') as logfile:
            logfile.write('# timestamp\t')
            logfile.write('# Name\t')
            for param in parameters:
                logfile.write(param + '\t')
            logfile.write('\n#\n')
    with open(log_path, 'a') as logfile:
        logfile.write(timestamp + '\t')
        logfile.write(name + '\t')
        for param in parameters:
            logfile.write('{0:3.6e}\t'.format(kwargs[param]))
        logfile.write('\n')
    return

In [35]:
# Get the dictionary containing the default parameters for the desired sequence and alter the values as needed.
# generate_params = pulsedmasterlogic.generate_method_params['rabi']
generate_params=OrderedDict()
generate_params['name'] = 'HHtauseq'
generate_params['spinlock_amp'] = 0.125
generate_params['tau_start'] = 0.5e-6
generate_params['tau_step'] = 8.0e-6
generate_params['num_of_points'] = 30
generate_params['alternating'] = True

# Call the actual generation method using a wrapper method and pass the parameter set
pulsedmasterlogic.generate_predefined_sequence('Hartmann_Hahn_tau_sequence', generate_params)
# Give it a moment to generate
time.sleep(0.5)

# The created object is a PulseBlockEnsemble instance, a recipe to create a waveform.
# You can get all created PulseBlockEnsembles using this property (keys are ensemble names, values are the object instances):
#for ensemble_name in pulsedmasterlogic.saved_pulse_block_ensembles.keys():
#    print(ensemble_name)

# The "with_load" flag can be used to also load the waveform into the channels after creating it
pulsedmasterlogic.sample_sequence('HHtauseq', with_load=True)
# Wait for the sampling to finish. There is a status dictionary that can be used to check for process status.
while pulsedmasterlogic.status_dict['sampload_busy']:
    time.sleep(0.1)
    
pulsedmasterlogic.set_measurement_settings(invoke_settings=False, 
                                           controlled_variable= generate_params['tau_start'] + 
                                           np.arange(generate_params['num_of_points'])*generate_params['tau_step'],
                                           number_of_lasers=2*generate_params['num_of_points'], 
                                           laser_ignore_list=[], 
                                           alternating=True, 
                                           units=('s', 'arb. u.'))
time.sleep(0.5)

pulsedmasterlogic.set_fast_counter_settings(record_length=4.0e-6, number_of_gates=generate_params['num_of_points'])
time.sleep(0.5)

import winsound
frequency = 300  # Set Frequency To 2500 Hertz
duration = 1000  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)
# You can get a read-only view of the current measurement settings by using this property:
# for setting, value in pulsedmasterlogic.measurement_settings.items():
#     print('{0}:\n  {1}'.format(setting, value))

In [29]:
poi=poimanager.active_poi.get_key()
NV_name =poimanager.poi_list[poi]._name

In [36]:
refocus_poi(poi)

133924.66666666666 2.783913884411096e-08


True

In [31]:
pulsedmasterlogic.set_timer_interval(3)

In [37]:
measurement_time = 3*60*60
refocus_interval = 2*60

In [38]:
do_hh_tau_refocus(poi, measurement_time, refocus_interval)

133924.66666666666 4.728734381559775e-08
133924.66666666666 1.2256249657665051e-08
133924.66666666666 9.152952585540602e-09
133924.66666666666 1.0644083467240615e-08
133924.66666666666 6.507118215253569e-09
133924.66666666666 6.824918777067659e-09
133924.66666666666 9.22207779234362e-09
133924.66666666666 5.370312829655929e-09
133924.66666666666 3.41396053055483e-09
133924.66666666666 2.57001747297278e-09
133924.66666666666 1.43848911251946e-08
133924.66666666666 3.803653880679939e-09
133924.66666666666 5.8039462926555275e-09
2D gaussian fit not successfull
133924.66666666666 0.0
133924.66666666666 1.2163562981951983e-08
133924.66666666666 9.028402125897626e-09
133924.66666666666 8.483039095000667e-09
133924.66666666666 6.704272259434875e-09
133924.66666666666 1.4154780368706328e-09
133924.66666666666 1.0005060184327123e-09
133924.66666666666 5.81033121749128e-09
133924.66666666666 4.488905833307209e-09
133924.66666666666 1.240045195296302e-08
133924.66666666666 1.7875711120519187e-09


True

In [39]:
# save measurement
pulsedmasterlogic.save_measurement_data(tag='HHtauseq_'+str(generate_params['spinlock_amp'])+'_'+NV_name, with_error=True)

In [26]:
do_hh_tau(refocus_interval, 'HHtauseq', 'HHtauseq')

True

In [1]:
def refocus_poi(poi=None, max_drift=0.5, min_counts=10e+3):
    if poi==None:
        return False
    scannerlogic.stop_scanning()
    poimanager.go_to_poi(poi)
    pos_start = scannerlogic.get_position()
    NV_name =poimanager.poi_list[poi]._name
    poimanager.optimise_poi(poi)
    while optimizer.module_state() == 'idle':
        time.sleep(0.2)
    while optimizer.module_state() != 'idle':
        time.sleep(0.2)
    time.sleep(5)
    pos_end=scannerlogic.get_position()
    print(counterlogic.countdata.mean(),abs(pos_end[0]-pos_start[0]))
    if (abs(pos_end[0]-pos_start[0])>max_drift or abs(pos_end[1]-pos_start[1])>max_drift or 
       abs(pos_end[2]-pos_start[2])>max_drift or counterlogic.countdata.mean()<min_counts):
        scannerlogic.set_position('',x=pos_start[0],y=pos_start[1],z=pos_start[2])
        poimanager.set_new_position(poi,pos_start)
        print(abs(pos_end[0]-pos_start[0])>max_drift or abs(pos_end[1]-pos_start[1])>max_drift or 
       abs(pos_end[2]-pos_start[2])>max_drift or counterlogic.countdata.mean()<min_counts)
        return False
        write_to_logfile('PositionRefocus_log', str(datetime.datetime.now()),name= NV_name, 
                     x_pos=pos_start[0], y_pos=pos_start[1], z_pos=pos_start[2],sucess=0)
    write_to_logfile('PositionRefocus_log', str(datetime.datetime.now()),name= NV_name, 
                     x_pos=pos_end[0], y_pos=pos_end[1], z_pos=pos_end[2],sucess=1)
    return True

def do_hh_tau(m_time, save_tag='', stash_raw_data_tag = ''):
    
    pulsedmeasurementlogic.measurement_tag = save_tag
    
    # If everything is properly set, we can start a measurement simply by calling:
    pulsedmasterlogic.toggle_pulsed_measurement(True, stash_raw_data_tag)
    # Wait until the pulsedmeasurementlogic is actually busy and the measurement is running
    while pulsedmeasurementlogic.module_state() != 'locked':
        time.sleep(0.2)
        
    user_terminated = False
    start_time = time.time()
    while time.time() - start_time < m_time:
        if pulsedmeasurementlogic.module_state()=='idle':
            user_terminated = True
            time.sleep(0.2)    
            break
        time.sleep(0.5)
        
    pulsedmasterlogic.toggle_pulsed_measurement(False, stash_raw_data_tag)
    # Wait until the pulsedmeasurementlogic is actually idle and the measurement is stopped
    while pulsedmeasurementlogic.module_state() == 'locked':
        time.sleep(0.2)
    
    if save_tag=='':
        pulsedmasterlogic.save_measurement_data(tag='HHtauseq_'+str(generate_params['spinlock_amp'])+'_'+NV_name, with_error=True)
    time.sleep(2)
    return user_terminated

def do_hh_tau_refocus(poi, measurement_time, refocus_interval):
    pulsedmeasurementlogic._saved_raw_data.clear()
    end_measure=False
    xy16_total_time = measurement_time
    xy16_runtime = 0.0
    measurement_time = refocus_interval
    end_measure = do_hh_tau(refocus_interval, 'HHtauseq', 'HHtauseq')
    xy16_runtime += refocus_interval
    while xy16_total_time > xy16_runtime:
        for i in range(1):
            end_measure = not refocus_poi(poi)
            if not end_measure:
                break
        if end_measure:
            break
        end_measure = do_hh_tau(refocus_interval, 'HHtauseq', 'HHtauseq')
        xy16_runtime += refocus_interval
        if end_measure:
            break
    write_to_logfile('HHtauseq', str(datetime.datetime.now()),name= NV_name, runtime=xy16_runtime)
    time.sleep(2)
    return end_measure